In [1]:
import numpy
from sklearn.preprocessing import minmax_scale
from sklearn.model_selection import train_test_split
from keras.layers import Input, Dense
from keras.models import Model
import pickle
from sklearn.preprocessing import *
from sklearn.svm import *
from sklearn.model_selection import *
from matplotlib import pyplot as plt
import os
import time
import numpy
import pickle
import cProfile
import itertools
import matplotlib
import sklearn.tree
import sklearn.metrics
import sklearn.ensemble
import sklearn.preprocessing
import sklearn.learning_curve
import sklearn.model_selection
import sklearn.cross_validation
import sklearn.feature_selection
import sklearn.kernel_approximation
from matplotlib import pyplot as plt
from sklearn.ensemble import IsolationForest
from sklearn.cross_validation import *
from sklearn.metrics import *
from sklearn.feature_selection import *
from sklearn.preprocessing import *
from sklearn.svm import *

%matplotlib inline
matplotlib.use('Agg')

#Loads pickled dataset
inFile = open('dataWithMet.pkl', 'rb')
dataset = pickle.load(inFile, encoding="latin1")
inFile.close()

inFile = open('jetMetTarget.pkl', 'rb')
target = pickle.load(inFile, encoding="latin1")
inFile.close()

# lumi = dataset[:,-1]
target[target == 0] = -1
scaler = StandardScaler()
dataset = scaler.fit_transform(dataset)

#xTrain has 90percent of dataset without labels and yTrain is their labels
#xTest has 10percent of dataset without labels and yTest is their labels

xTrain, xTest, yTrain, yTest = train_test_split(dataset, target, test_size=.1, random_state=1)

Using TensorFlow backend.
/Users/fsiroky/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/fsiroky/anaconda3/lib/python3.6/site-packages/sklearn/learning_curve.py:23: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)
/Users/fsiroky/anaconda3/lib/python3.6/site-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplo

In [2]:
input_dim = Input(shape = (43, ))
# DEFINE THE DIMENSION OF ENCODER ASSUMED 15
encoding_dim = 20
# DEFINE THE ENCODER LAYERS
encoded1 = Dense(38, activation = 'relu')(input_dim)
encoded2 = Dense(35, activation = 'relu')(encoded1)
encoded3 = Dense(32, activation = 'relu')(encoded2)
encoded4 = Dense(28, activation = 'relu')(encoded3)
encoded5 = Dense(24, activation = 'relu')(encoded4)
encoded6 = Dense(encoding_dim, activation = 'relu')(encoded5)
# DEFINE THE DECODER LAYERS
decoded1 = Dense(24, activation = 'relu')(encoded6)
decoded2 = Dense(28, activation = 'relu')(decoded1)
decoded3 = Dense(32, activation = 'relu')(decoded2)
decoded4 = Dense(35, activation = 'relu')(decoded3)
decoded5 = Dense(38, activation = 'relu')(decoded4)
decoded6 = Dense(43, activation = 'sigmoid')(decoded5)

# COMBINE ENCODER AND DECODER INTO AN AUTOENCODER MODEL
autoencoder = Model(input = input_dim, output = decoded6)
# CONFIGURE AND TRAIN THE AUTOENCODER
autoencoder.compile(optimizer = 'Adam', loss = 'binary_crossentropy') #Adamax loss = 0.4366,
#Adagrad = 0.4306, Adadelta = 0.4286, RMSprop = 0.4275, Adam = 0.4274
autoencoder.fit(xTrain, xTrain, nb_epoch = 10, batch_size = 128, shuffle = True, validation_data = (xTest, xTest ))
# THE ENCODER TO EXTRACT THE REDUCED DIMENSION FROM THE ABOVE AUTOENCODER
encoder = Model(input = input_dim, output = encoded6)
encoded_input = Input(shape = (encoding_dim, ))
encoded_out = encoder.predict(xTrain)
print(encoded_out)

/Users/fsiroky/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:20: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
/Users/fsiroky/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:24: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 209194 samples, validate on 23244 samples
Epoch 1/10
209194/209194 [==============================] - 14s - loss: -3.3595 - val_loss: -3.9927

/Users/fsiroky/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:26: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


[[  8.21784401   0.95769113   0.         ...,   0.           7.12905264
    4.50031376]
 [ 10.41616154   0.           0.         ...,   0.          18.11907959
    7.46096373]
 [  9.45774078   0.           0.         ...,   0.          18.61679077
    7.97259331]
 ..., 
 [  6.55435038   0.           0.         ...,   0.          14.02589893
    2.05475903]
 [  7.44896507   0.90164107   0.         ...,   0.          14.76055908
    6.32737446]
 [ 14.35646152   7.34467077   0.         ...,   0.           9.76404572
    0.        ]]
